In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Rosbank ML Competition

Ссылка на соревнование: https://boosters.pro/champ_15

Росбанк – часть ведущей международной финансовой группы Societe Generale, банк включен ЦБ РФ в число 11 системно значимых кредитных организаций России. Инновации неотъемлемый процесс работы Росбанка, поэтому активно развивается направленный анализа больших данных.

- Данные

Датасет, который содержит историю транзакций клиентов за 3 месяца льготного использования банковского продукта

- Задача

Задача бинарной классификации – прогноз оттока клиентов

Колонка cl_id содержит вутренний id клиента. Для каждого уникальнго cl_id следует предсказать продолжит ли клиент пользоваться продуктом (target_flag). Значение 0 соответствует отказу, а значение 1 соответствует продолжению использования

In [224]:
train = pd.read_csv('train.csv')
train.head(5)

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0


In [225]:
test = pd.read_csv('test.csv')
test.head(5)

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category
0,01/04/2017,2,5814,NaN,810,18APR17:00:00:00,153.00,POS
1,01/03/2017,2,5812,NaN,810,10MAR17:00:00:00,388.08,POS
2,01/02/2017,2,5691,NaN,810,21FEB17:00:00:00,999.00,POS
3,01/04/2017,2,5814,NaN,810,26APR17:00:00:00,90.00,POS
4,01/04/2017,2,5814,NaN,810,08APR17:00:00:00,130.00,POS


In [226]:
train.drop('target_sum', axis=1, inplace=True)

In [227]:
test['target_flag'] = "y"

In [228]:
train.columns == test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [229]:
df = pd.concat([train, test])

In [ ]:
def get_amount_sum(r):
    return r.amount.sum()

In [ ]:
def get_main_currency(r):
    return(r.currency.value_counts().index.tolist()[0])

In [230]:
data_full = pd.DataFrame({
        'amount_sum' : df.groupby( [ "cl_id", "target_flag" ] ).apply(get_amount_sum),
        'main_currency' :  df.groupby( [ "cl_id", "target_flag" ] ).apply(get_main_currency)
    }).reset_index()

In [232]:
data_full.head(10)

,cl_id,target_flag,amount_sum,main_currency
0,0,0,64383.00,810
1,1,0,267578.04,810
2,2,y,328717.72,810
3,3,y,270848.77,810
4,4,y,338177.98,810
5,5,1,546369.13,810
6,6,y,460876.84,810
7,7,y,194243.43,810
8,8,y,303505.35,810
9,9,0,849315.09,810


In [233]:
X_train = data_full[data_full.target_flag=='y'].drop('target_flag', axis=1)
X_test = data_full[data_full.target_flag!='y'].drop('target_flag', axis=1)

In [236]:
Y_train = data_full[data_full.target_flag!='y']['target_flag']

In [237]:
from sklearn.linear_model import

0        0
1        0
5        1
9        0
10       0
11       0
14       1
20       0
21       0
22       0
23       1
24       1
25       1
28       1
29       1
31       1
32       1
35       1
38       0
39       0
41       1
42       0
46       0
48       1
49       0
50       1
52       0
54       0
55       0
56       1
        ..
10162    1
10163    0
10164    1
10166    1
10167    1
10168    0
10169    0
10170    1
10171    0
10175    0
10176    1
10177    0
10180    0
10181    0
10184    1
10185    1
10186    1
10188    1
10190    0
10192    1
10193    1
10202    1
10203    0
10206    0
10207    1
10210    1
10212    0
10213    0
10214    0
10215    0
Name: target_flag, Length: 5000, dtype: object

In [157]:
data_short  = data[:15]

In [210]:
def get_amount_sum(r):
   # print(r)
    #print(r.amount.sum())
    return r.amount.sum()

In [211]:
def get_main_currency(r):
    return(r.currency.value_counts().index.tolist()[0])

In [221]:
d = pd.DataFrame({
        'amount_sum' : data_short.groupby( [ "cl_id", "target_flag" ] ).apply(get_amount_sum),
        'main_currency' :  data_short.groupby( [ "cl_id", "target_flag" ] ).apply(get_main_currency)
    }).reset_index()

In [222]:
d

,cl_id,target_flag,amount_sum,main_currency
0,0,0,64383.0,810
1,1,0,9151.9,810


## Домашняя работа

1. Наборы данных вида Transactions (несколько транзакций на одного клиента) трансформировать в таблицу, где cl_id будут уникальными (соответственно 4000 строк в train и 1000 строк в test
2. Для каждого cl_id будет уникальное целевое событие target_flag, а также уникальный канал привлечения клиента channel_type (клиент привлекается лишь однажды и с самого начала его записи присваивается значение канала привлечения)
3. При агрегации (*pandas.DataFrame.groupby*) по cl_id (или по связке cl_id, channel_type, target_flag) необходимо создавать производные фичи, идеи для таких фичей могут быть следующими:

    - общая сумма транзакций по каждой из trx_category
    - общая сумма транзакции по основным вылютам (напр. выделить рубли, доллары и евро - предположительно, это будут самые крупные категории)
    - общая сумма транзакций по категориям MCC кодов (например, выбрать основные/популярные MCC коды). ВНИМАНИ! Некоторые MCC коды из train могут быть не представлены в test. Про MCC коды в целом: http://www.banki.ru/wikibank/mcc-kod/; Справочник MCC кодов: https://mcc-codes.ru/code; Про некоторые категории кэшбека Росбанка: https://mcc-codes.ru/card/rosbank-sverkh-plus;
    - возможные агрегации по времени суток и дням недели - траты в выходные (праздники) или будни, в ночное время или в рабочее и т.д.
3. **Обязательная часть**: провести первичный анализ данных - посмотреть распределения признаков, выделить самые популярные MCC, помотреть активность клиентов по дням недели/времени, какие категории транзакции (trx_category) наиболее популярны и т.д. Получить инсайты, которые в дальнейшем помогут вам правильно подготовить фичи
4. **Обязательная часть**: на большом количестве фичей применить Lasso регрессию и посмотреть, какие переменные получают 0 в качестве коэффициентов
5. **Обязательная часть**: на фичах из MCC кодов применить PCA и посмотреть, какие фичи получаются и что они могут объяснять (для этого посмотреть на коэффициенты в *sklearn.decomposition.pca.components_* после построения главных компонент)
6. **Обязательная часть**: с отобранными фичами и полученными компонентами обучить модель (тип алгоритма на свой вкус, можно начать с линейной) и померить качество на локальном тестовом наборе данных (локальная валидация), который создается в этом ноутбуке. Сравнить качество модели до добавления компонент/после добавления/только на главных компонентах. **Метрика оценки качества - ROC AUC**(https://en.wikipedia.org/wiki/Receiver_operating_characteristic)
8. **Дополнительная часть**: поучаствовать в соревновании - загрузить предсказания на https://boosters.pro/champ_15. Для этого необходимо использовать все данные из файла rosbank_train.csv, на них делать feature engineering и обучениеб затем делать предсказания для клиентов из файла rosbank_test.csv, предварительно создав фичи по аналогии с train.
9. Задания принимаются в виде ноутбука с кодом/картинками выполненной обязательной части + указанием места в leaderboard при решении дополнительной

При возникновении вопросов и для отправки домашнего задания - egsachko@gmail.com или http://fb.com/sachkoe
    

In [9]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                         'foo', 'bar', 'foo', 'foo'],
                  'B' : ['one', 'one', 'two', 'three',
                         'two', 'two', 'one', 'three'],
                   'C' : np.random.randn(8),
                     'D' : np.random.randn(8)})
   

In [10]:
df

,A,B,C,D
0,foo,one,1.837962,0.736571
1,bar,one,0.094909,1.237023
2,foo,two,-0.108882,-0.838908
3,bar,three,0.316443,-0.210513
4,foo,two,0.181880,-0.312836
5,bar,two,-2.689405,-2.475270
6,foo,one,0.265598,0.992919
7,foo,three,-0.112345,1.147616


In [11]:
grouped = df.groupby('A')

In [12]:
grouped

In [13]:
grouped = df.groupby(['A', 'B'])

In [14]:
grouped

In [15]:
def get_letter_type(letter):
        if letter.lower() in 'aeiou':
            return 'vowel'
        else:
            return 'consonant'

In [16]:
grouped = df.groupby(get_letter_type, axis=1)

In [17]:
grouped

In [75]:

ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
         'kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
         'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
         'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
         'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}
df = pd.DataFrame(ipl_data)

grouped = df.groupby('Year')
print(grouped)
for name,group in grouped:
    print(name)
    print(group)

2014
   Points  Rank    Team  Year
0     876     1  Riders  2014
2     863     2  Devils  2014
4     741     3   Kings  2014
9     701     4  Royals  2014
2015
    Points  Rank    Team  Year
1      789     2  Riders  2015
3      673     3  Devils  2015
5      812     4   kings  2015
10     804     1  Royals  2015
2016
   Points  Rank    Team  Year
6     756     1   Kings  2016
8     694     2  Riders  2016
2017
    Points  Rank    Team  Year
7      788     1   Kings  2017
11     690     2  Riders  2017
